In [105]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
train = pd.read_csv("../input/predict-volcanic-eruptions-ingv-oe/train.csv")
train.head()

,segment_id,time_to_eruption
0,1136037770,12262005
1,1969647810,32739612
2,1895879680,14965999
3,2068207140,26469720
4,192955606,31072429


In [7]:
train_dir = "../input/predict-volcanic-eruptions-ingv-oe/train/"
test_dir = "../input/predict-volcanic-eruptions-ingv-oe/test/"

In [8]:
def read_csv(index):
    train1 = pd.read_csv(train_dir + str(train.segment_id.iloc[index]) + ".csv")
    train1['timetoerupt'] = train.time_to_eruption.iloc[index]
    for feat in train1.drop('timetoerupt',1).columns:
        train1[feat] = train1[feat].mean()
    train1 = train1.sample(1)
    return (train1)

In [9]:
# add all the training data sets into one dataframe
data = pd.DataFrame()
for idx in range(train.shape[0]):
    df = read_csv(idx)
    data=pd.concat([df,data])

In [11]:
# load training data
data.head()

,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,sensor_10,timetoerupt
52203,-4.142848,0.138031,1.955217,0.241129,-1.577340,0.385694,1.526325,0.802537,-1.700145,-45.584790,20128938
54163,0.905818,NaN,-0.648256,-2.339661,0.320345,6.707488,-3.654656,NaN,1.104332,-6.360564,9598270
20898,1.420993,0.446893,-1.197830,1.510475,-0.514108,-1.531391,1.858136,1.647323,-1.149198,-1.550374,31206935
49134,7.936884,NaN,5.014233,-4.597357,NaN,0.607590,1.506825,-4.511225,5.498189,10.939634,35659379
10219,0.906735,-3.658256,-0.751437,1.915551,-1.734333,-22.033699,1.607623,0.046333,-1.802253,-3.377394,15695097


In [12]:
data.shape

(4431, 11)

In [13]:
data.isnull().sum()

sensor_1        24
sensor_2       835
sensor_3       435
sensor_4         0
sensor_5       595
sensor_6         0
sensor_7        42
sensor_8       440
sensor_9       124
sensor_10       24
timetoerupt      0
dtype: int64

In [14]:
# replace null values with the mean value
for feat in data:
    data[feat] = data[feat].replace(np.nan, data[feat].mean())

In [15]:
data.isnull().sum()

sensor_1       0
sensor_2       0
sensor_3       0
sensor_4       0
sensor_5       0
sensor_6       0
sensor_7       0
sensor_8       0
sensor_9       0
sensor_10      0
timetoerupt    0
dtype: int64

In [16]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [25]:
X_train, X_test, y_train, y_test = train_test_split(data, data.timetoerupt, test_size=0.2, random_state=42)

In [26]:
X_train.drop('timetoerupt',1,inplace = True)

# Add a constant to get an intercept
X_train_sm = sm.add_constant(X_train)

# train the model
lr = sm.OLS(y_train, X_train_sm).fit()

In [27]:
print(lr.summary())

                            OLS Regression Results                            
Dep. Variable:            timetoerupt   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     2.785
Date:                Sat, 28 Nov 2020   Prob (F-statistic):            0.00196
Time:                        20:20:39   Log-Likelihood:                -63197.
No. Observations:                3544   AIC:                         1.264e+05
Df Residuals:                    3533   BIC:                         1.265e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.293e+07   2.27e+05    101.243      0.0

In [28]:
X_test.drop('timetoerupt',1,inplace = True)

# Add a constant to get an intercept
X_test_sm = sm.add_constant(X_test)

# prediction on training dataset
y_test_pred = lr.predict(X_test_sm)

In [29]:
r_squared = r2_score(y_test_pred, y_test)
r_squared

-486.86449175842216

In [83]:
sub = pd.read_csv("../input/predict-volcanic-eruptions-ingv-oe/sample_submission.csv")
sub.head()

,segment_id,time_to_eruption
0,1000213997,0
1,100023368,0
2,1000488999,0
3,1001028887,0
4,1001857862,0


In [31]:
def read_csv(index):
    
    test1 = pd.read_csv(test_dir + str(sub.segment_id.iloc[index]) + ".csv")

    for feat in test1.columns:
        test1[feat] = test1[feat].mean()
    
    test1 = test1.sample(1)
           
    return (test1)

In [32]:
test = pd.DataFrame()
for idx in range(sub.shape[0]):
    df = read_csv(idx)
    test = pd.concat([df,test])

In [61]:
# load the testing dataset
test.head()

,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,sensor_10
47341,-2.715638,1.782228,-3.888219,1.855736,0.258996,5.089865,5.543958,4.414276,1.950851,0.000000
2829,3.218563,-0.552941,-2.343444,-2.133098,0.155914,-0.957801,-1.103298,-2.347594,0.805911,9.977984
20124,1.192280,-0.885119,4.310145,0.304878,1.015366,14.487142,-1.014200,-0.774304,-0.373064,-0.077015
59627,-2.666439,1.447959,-2.014883,0.893418,1.015366,3.223130,0.324595,7.570357,0.598209,3.488892
8740,-135.674672,-999.268329,-72.957951,-624.627090,1.015366,-198.286962,-96.878002,-230.499275,-0.373064,-0.077015


In [62]:
# again verify whether all the files were read correctly or not
test.shape

(4520, 10)

In [36]:
test.isnull().sum()

sensor_1      387
sensor_2     1257
sensor_3      434
sensor_4        0
sensor_5     1388
sensor_6        0
sensor_7       34
sensor_8      294
sensor_9      817
sensor_10     760
dtype: int64

In [37]:
# replace null values with mean, same as the training data
for feat in test:
    test[feat] = test[feat].replace(np.nan, test[feat].mean())

In [38]:
test.isnull().sum()

sensor_1     0
sensor_2     0
sensor_3     0
sensor_4     0
sensor_5     0
sensor_6     0
sensor_7     0
sensor_8     0
sensor_9     0
sensor_10    0
dtype: int64

In [95]:
# Add a constant to get an intercept
test_sm = sm.add_constant(test)

# prediction on test dataset
predictions = lr.predict(test_sm)
sub['time_to_eruption'] = predictions.values

In [96]:
sub.head()

,segment_id,time_to_eruption
0,1000213997,2.276998e+07
1,100023368,2.305317e+07
2,1000488999,2.286145e+07
3,1001028887,2.304319e+07
4,1001857862,2.309563e+07


In [97]:
# submission file
sub.to_csv('submission.csv',index=False)